PART II: HPA CELL SEGEMENTATION WITH MULTIPLE CHANNELS
-------------------------------------

NOTES: 
- SubCell was trained on individual cell crops from the Human Protein Atlas (HPA) SubCellular data, which includes immunofluorescence of 13,147 proteins of interest and 37 different human cell lines. Below are example field of view images for each of the 4 channels in the 2D HPA data: endoplasmic reticulum (yellow), nucleus (blue), microtubules (red), and protein of interest (green).

Script Summary: 
- Matches red/yellow/blue images by basename across folders.
- Converts each to grayscale, stacks into a 3-channel (R,Y,B) image.
- Auto-downloads nuclei/cell model weights if missing and selects CPU/GPU.
- Runs hpacellseg for nuclei and cell segmentation (multi-channel).
- Saves nuclei/cell masks (PNG) and raw predictions (NPY) + a summary in analysis/segmentation_results2.

*See end of document for detailed script summary.

In [1]:
#INSTALL PACKAGES, IMAGING DEPS AND CLONES HPA_CELL_SEGMENTATION
import os
import sys
import glob
from pathlib import Path
import numpy as np
from PIL import Image
import imageio

!pip3 install scikit-image imageio scipy opencv-python pillow==6.2.1
!pip install git+https://github.com/haoxusci/pytorch_zoo@master#egg=pytorch_zoo
!pip3 install --upgrade torch

#!git clone https://github.com/CellProfiling/HPA-Cell-Segmentation.git
#%cd HPA-Cell-Segmentation

!sh install.sh
!python -c "import hpacellseg"

  Cloning https://github.com/haoxusci/pytorch_zoo (to revision master) to c:\users\bernalr3\appdata\local\temp\pip-install-6x_qz23l\pytorch-zoo_e783301dfd18458ab71d757343b91835
  Resolved https://github.com/haoxusci/pytorch_zoo to commit e7f30cd9d2e900439f98d250ac309caf03a166b4
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/haoxusci/pytorch_zoo 'C:\Users\bernalr3\AppData\Local\Temp\pip-install-6x_qz23l\pytorch-zoo_e783301dfd18458ab71d757343b91835'


sh: install.sh: No such file or directory


In [2]:
#Cloning
!git clone https://github.com/CellProfiling/HPA-Cell-Segmentation.git
%cd HPA-Cell-Segmentation

c:\Users\bernalr3\OneDrive - UT Health San Antonio\Archive\Documents\CM4AI - WG\analysis\HPA-Cell-Segmentation


fatal: destination path 'HPA-Cell-Segmentation' already exists and is not an empty directory.


In [ ]:
#!/usr/bin/env python3
"""
HPA-Cell-Segmentation Script (multi-channel, RGB->grayscale fix)
- Matches red (microtubules), yellow (ER), blue (nuclei) by normalized basename
- Converts each channel image to grayscale and stacks as HxWx3 (R,Y,B)
- Runs nuclei + cell segmentation and saves masks/predictions

Assumes folders:
  DATA_ROOT/red, DATA_ROOT/yellow, DATA_ROOT/blue
The green channel (protein) is ignored by hpacellseg.

Edit the USER PATHS below to match your machine.
"""

import re
import sys
from pathlib import Path

import numpy as np
from PIL import Image
import imageio

# ------------ Quiet noisy dependency warnings (optional, safe) ------------
import warnings
warnings.filterwarnings("ignore", message=".*multichannel.*", category=FutureWarning)
try:
    from torch.serialization import SourceChangeWarning
    warnings.filterwarnings("ignore", category=SourceChangeWarning)
except Exception:
    pass
warnings.filterwarnings("ignore", message=".*weights_only=False.*", category=FutureWarning)
# --------------------------------------------------------------------------

# ---- NumPy compatibility shim (NumPy >=1.24 removed legacy aliases) ----
if "bool"  not in np.__dict__: np.bool  = np.bool_
if "int"   not in np.__dict__: np.int   = np.int_
if "float" not in np.__dict__: np.float = np.float64
# ------------------------------------------------------------------------

# ===================== USER PATHS (update these) =====================
HPA_PATH     = Path(r"C:\Users\bernalr3\OneDrive - UT Health San Antonio\Archive\Documents\HPA-Cell-Segmentation")
DATA_ROOT    = Path(r"C:\Users\bernalr3\OneDrive - UT Health San Antonio\Archive\Documents\CM4AI - WG\data")
ANALYSIS_DIR = Path(r"C:\Users\bernalr3\OneDrive - UT Health San Antonio\Archive\Documents\CM4AI - WG\analysis")
# =====================================================================

# Try import from installed package; otherwise import from local clone
try:
    import hpacellseg.cellsegmentator as cellsegmentator
    from hpacellseg.utils import label_nuclei, label_cell
except ImportError:
    if (HPA_PATH / "hpacellseg").exists():
        sys.path.insert(0, str(HPA_PATH))
        import hpacellseg.cellsegmentator as cellsegmentator
        from hpacellseg.utils import label_nuclei, label_cell
    else:
        raise ImportError(
            "Cannot import hpacellseg. Either install it with:\n"
            f'  pip install -e "{HPA_PATH}"\n'
            "or update HPA_PATH to your clone."
        )

def list_images(folder: Path):
    exts = ('.tif', '.tiff', '.png', '.jpg', '.jpeg')
    return [p for p in folder.iterdir() if p.is_file() and p.suffix.lower() in exts]

# Strip common channel tokens so files can be matched by basename
CHANNEL_TOKENS = re.compile(
    r'(?i)(?:^|[_\-\s])('
    r'red|r|microtubules?|mtub|tubulin|tub|'
    r'yellow|yel|y|er|reticulum|'
    r'blue|b|nuclei|nuc|dapi|dna|'
    r'ch(?:an(?:nel)?)?_?\d{1,2}'
    r')(?=$|[_\-\s])'
)

def normalize_key(path: Path):
    s = path.stem.lower()
    s = CHANNEL_TOKENS.sub('', s)
    s = re.sub(r'[_\-\s]+', '_', s).strip('_')
    return s

def natural_key(s: str):
    return [int(t) if t.isdigit() else t for t in re.split(r'(\d+)', s)]

def read_gray(path: Path, target_shape=None):
    """Load image, convert to grayscale (L), return 2D uint8; resize to target_shape=(H,W) if provided."""
    img = Image.open(path).convert('L')  # force single channel
    if target_shape is not None and img.size != (target_shape[1], target_shape[0]):  # PIL size=(W,H)
        img = img.resize((target_shape[1], target_shape[0]), resample=Image.BILINEAR)
    return np.array(img)

def main():
    ANALYSIS_DIR.mkdir(parents=True, exist_ok=True)
    out_dir = ANALYSIS_DIR / "segmentation_results2"
    out_dir.mkdir(parents=True, exist_ok=True)

    red_dir    = DATA_ROOT / "red"     # microtubules
    yellow_dir = DATA_ROOT / "yellow"  # ER
    blue_dir   = DATA_ROOT / "blue"    # nuclei (DAPI)

    for d in (red_dir, yellow_dir, blue_dir):
        d.mkdir(parents=True, exist_ok=True)

    red_list    = list_images(red_dir)
    yellow_list = list_images(yellow_dir)
    blue_list   = list_images(blue_dir)

    print(f"Counts — red: {len(red_list)}, yellow: {len(yellow_list)}, blue: {len(blue_list)}")

    # Build maps by normalized key
    def build_map(paths):
        m = {}
        for p in paths:
            k = normalize_key(p)
            m.setdefault(k, []).append(p)
        return m

    red_map    = build_map(red_list)
    yellow_map = build_map(yellow_list)
    blue_map   = build_map(blue_list)

    common_keys = sorted(set(red_map) & set(yellow_map) & set(blue_map), key=natural_key)
    if not common_keys:
        print("No matched (red, yellow, blue) triplets after normalization.")
        # Debug a few keys to help user adjust tokens
        for name, m in (("red", red_map), ("yellow", yellow_map), ("blue", blue_map)):
            print(f"Sample normalized keys for {name}:", list(m.keys())[:5])
        return

    print(f"Matched {len(common_keys)} triplets (red/yellow/blue).")

    # Prepare grayscale arrays and precombined HxWx3 stacks (R=red, Y=yellow, B=blue)
    combined_cells = []  # list of HxWx3 uint8
    nuc_grays      = []  # list of HxW uint8
    stems          = []

    for k in common_keys:
        r_path = sorted(red_map[k],    key=lambda p: natural_key(p.name))[0]
        y_path = sorted(yellow_map[k], key=lambda p: natural_key(p.name))[0]
        b_path = sorted(blue_map[k],   key=lambda p: natural_key(p.name))[0]

        # Read nuclei first to set target size
        b_gray = read_gray(b_path)
        r_gray = read_gray(r_path, target_shape=b_gray.shape)
        y_gray = read_gray(y_path, target_shape=b_gray.shape)

        combo = np.stack([r_gray, y_gray, b_gray], axis=2)  # HxWx3
        combined_cells.append(combo)
        nuc_grays.append(b_gray)
        stems.append(k if k else b_path.stem)

    # Model weights (auto-download if missing; delete zero-byte placeholders)
    nuclei_model_path = ANALYSIS_DIR / "nuclei_model.pth"
    cell_model_path   = ANALYSIS_DIR / "cell_model.pth"
    for p in (nuclei_model_path, cell_model_path):
        if p.exists() and p.stat().st_size < 1024:
            print(f"{p.name} looks like a zero-byte placeholder; deleting to trigger re-download.")
            p.unlink()

    # Device
    try:
        import torch
        device = "cuda" if torch.cuda.is_available() else "cpu"
    except Exception:
        device = "cpu"
    print("Using device:", device)

    # Initialize multi-channel model
    print("Initializing CellSegmentator (multi_channel_model=True)...")
    segmentator = cellsegmentator.CellSegmentator(
        str(nuclei_model_path),
        str(cell_model_path),
        scale_factor=0.25,
        device=device,
        padding=True,
        multi_channel_model=True
    )
    print("CellSegmentator initialized")

    # Run segmentation
    print("Running nuclei predictions...")
    nuc_preds  = segmentator.pred_nuclei(nuc_grays)  # pass 2D arrays
    print("Running cell predictions (precombined RGB stacks)...")
    cell_preds = segmentator.pred_cells(combined_cells, precombined=True)  # <-- key change

    # Save results
    for i, stem in enumerate(stems, 1):
        print(f"\nPost-processing {i}/{len(stems)}: {stem}")

        nuc_mask = label_nuclei(nuc_preds[i-1])
        nuclei_mask, cell_mask = label_cell(nuc_preds[i-1], cell_preds[i-1])

        nuc_mask_png  = out_dir / f"{stem}_nuclei_mask.png"
        cell_mask_png = out_dir / f"{stem}_cell_mask.png"
        nuc_pred_npy  = out_dir / f"{stem}_nuclei_prediction.npy"
        cell_pred_npy = out_dir / f"{stem}_cell_prediction.npy"

        imageio.imwrite(str(nuc_mask_png),  nuc_mask.astype(np.uint16))
        imageio.imwrite(str(cell_mask_png), cell_mask.astype(np.uint16))
        np.save(str(nuc_pred_npy),  nuc_preds[i-1])
        np.save(str(cell_pred_npy), cell_preds[i-1])

        print(f"  Saved masks: {nuc_mask_png.name}, {cell_mask_png.name}")

    # Summary
    summary = out_dir / "segmentation_summary.txt"
    with open(summary, "w") as f:
        f.write("HPA-Cell-Segmentation Results Summary (multi-channel, grayscale-fixed)\n")
        f.write("=" * 70 + "\n\n")
        f.write(f"Matched triplets: {len(stems)}\n")
        f.write(f"Data root: {DATA_ROOT}\nOutput dir: {out_dir}\n")
        f.write("Channels used: red(microtubules)=R, yellow(ER)=Y, blue(nuclei)=B; green ignored.\n")
        f.write("Models: nuclei_model.pth, cell_model.pth\n")
        f.write(f"Scale factor: 0.25 | Device: {device.upper()} | Padding: True | Multi-channel: True\n")

    print(f"\nDone. Results in: {out_dir}")

if __name__ == "__main__":
    main()
### Results in: C:\Users\bernalr3\OneDrive - UT Health San Antonio\Archive\Documents\CM4AI - WG\analysis\segmentation_results2

Detailed Script Summary: 
1. Sets things up
- Points to your repo (HPA_PATH), data root (... \data), and output folder (... \analysis\segmentation_results2).
- Silences a few noisy warnings and adds tiny NumPy “compatibility shims” so old code runs on new NumPy.

2. Finds and matches your images
- Looks inside three folders under data: red/ (microtubules), yellow/ (ER), and blue/ (nuclei/DAPI).
- Normalizes filenames (e.g., strips _red, ER, DAPI, ch1, etc.) so it can pair the same field-of-view across channels.
- Builds a list of matched triplets (one red + one yellow + one blue per FOV).

3. Prepares inputs for the model
- Loads each file as grayscale (single channel).
- Resizes red and yellow to exactly match the blue image size.
- Stacks them into a single 3-channel array in this order: R = red, Y = yellow, B = blue (shape H×W×3).
- Also keeps the blue grayscale image alone for nuclei-only segmentation.

4. Gets model weights & device
- Points to nuclei_model.pth and cell_model.pth in your analysis folder; if a file is a tiny placeholder, deletes it so the package will auto-download it.
- Chooses CUDA if available, otherwise CPU.

5. Initializes the HPA segmentator
- Creates CellSegmentator(..., multi_channel_model=True) so it can use the 3-channel cell model.

6. Runs segmentation
- Nuclei: calls pred_nuclei(...) on the blue (nuclei) grayscale images.
- Cells: calls pred_cells(precombined=True) on the stacked R/Y/B arrays to get whole-cell predictions.

7. Post-processes & saves results
- Converts raw nuclei predictions to a labeled nuclei mask with label_nuclei(...).
- Combines nuclei + cell predictions to get a labeled cell mask with label_cell(...).
- Saves:
    *_nuclei_mask.png and *_cell_mask.png (PNG label images)
    *_nuclei_prediction.npy and *_cell_prediction.npy (raw model outputs)

- Writes a summary text file listing what was processed and the settings used.

8. What it ignores
- The green channel (protein of interest) isn’t used by hpacellseg for segmentation, so the script doesn’t load it.

9. Where results go
- Everything lands in:
    ...\CM4AI - WG\analysis\segmentation_results2